In [2]:
import subprocess
import tempfile
import os
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("LoadGHArchiveData").getOrCreate()

# Create a temp file manually and keep it until Spark reads it
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".json")
temp_file_path = temp_file.name  # Get the temp file path
temp_file.close()  # Close so subprocess can write to it

try:
    # Use subprocess to fetch and decompress data into the temp file
    with open(temp_file_path, "wb") as f:
        curl_process = subprocess.Popen(
            ["curl", "-s", "https://data.gharchive.org/2025-01-01-0.json.gz"],
            stdout=subprocess.PIPE
        )
        gunzip_process = subprocess.Popen(
            ["gunzip", "-c"], stdin=curl_process.stdout, stdout=f
        )
        gunzip_process.communicate()  # Wait for process to complete

    # Load JSON file into Spark DataFrame
    df = spark.read.json(temp_file_path)
    df.show(truncate=False)  # Display data

finally:
    # Clean up the temp file after Spark reads it
    os.remove(temp_file_path)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------